<a href="https://colab.research.google.com/github/Ricardojnf33/Spark_actionAD1/blob/main/Exercicio_Spark_AWS_Ricardo_Fernandes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalando as dependências

In [ ]:
!apt-get update

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:12 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [2,077 k

In [ ]:
# instalar as dependências
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark


# Criando variáveis de ambiente

In [ ]:
# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

# tornar o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

# Importando Bibliotecas

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import col, asc, desc, dayofweek
from pyspark.sql.functions import when

## Iniciando a sessão

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
from pyspark.sql.types import StructType, IntegerType, TimestampType, StringType, StructField

schema = StructType([
    StructField("marketplace",StringType()),
    StructField("customer_id",StringType()),
    StructField("review_id",StringType()),
    StructField("product_id",StringType()),
    StructField("product_parent",StringType()),
    StructField("product_title",StringType()),
    StructField("product_category",StringType()),
    StructField("star_rating",StringType()),
    StructField("helpful_votes",StringType()),
    StructField("total_votes",StringType()),
    StructField("vine",StringType()),
    StructField("verified_purchase",StringType()),
    StructField("review_headline ",StringType()),
    StructField("review_body",StringType()),
    StructField("review_date",StringType())
])

Upload do AWS

In [ ]:
!wget -i https://s3.amazonaws.com/amazon-reviews-pds/tsv/index.txt
!gunzip /content/amazon_reviews_us_*.tsv.gz

In [ ]:
#merge_tsv_files
from glob import glob

filename = 'merge.tsv'

with open(filename, 'a') as singleFile:
    first_tsv = True
    for tsv in glob('/content/amazon_reviews_us_*.tsv'):
        if tsv == filename:
            pass
        else:
            header = True
            for line in open(tsv, 'r'):
                if first_tsv and header:
                    singleFile.write(line)
                    first_tsv = False
                    header = False
                elif header:
                    header = False
                else:
                    singleFile.write(line)
    singleFile.close()

## Esvaziando o hard drive.

In [ ]:
!rm /content/amazon_reviews_*.tsv
!rm /content/amazon_reviews_*.tsv.gz

In [ ]:
df_ad = spark.read.csv('/content/merge.tsv', header=False, schema=schema, sep= "\t")
df_ad.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|    review_headline |         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
|         US|   24371595|R27ZP1F1CD0C3Y|B004LLIL5A|     346014806|Amazon eGift Card...|       Gift Card|          5|    

## 1) Quais são as top 3 campanhas que geraram mais eventos? Ordene pela quantidade de eventos (2,5 pontos)

In [ ]:
df_ad.groupby('product_category').count().orderBy(col('count').desc()).show(10)

+--------------------+--------+
|    product_category|   count|
+--------------------+--------+
|               Books|19531194|
|Digital_Ebook_Pur...|17622381|
|            Wireless| 9001881|
|                  PC| 6908551|
|                Home| 6221537|
|             Apparel| 5906322|
|Health & Personal...| 5331215|
|              Beauty| 5115452|
|           Video DVD| 5069136|
|         Mobile_Apps| 5033376|
+--------------------+--------+
only showing top 10 rows



## 2) Qual campanha teve mais clicks? (2,5 pontos)

In [ ]:
df_ad.where(df_ad.helpful_votes=='helpful_votes').groupby('review_id').count().orderBy(col('count').desc()).show(1)

+---------+-----+
|review_id|count|
+---------+-----+
|review_id|    1|
+---------+-----+



## 3) Dos 12 meses do ano, qual teve o maior total de eventos acumulado ao longo dos anos? (2,5 pontos)

In [ ]:
df_ad.where(df_ad.action=='click').groupby('campaignId').count().orderBy(col('count').desc()).show(1)

In [ ]:
df_ad.where(df_ad.action=='click').assign(month=lambda x: x['timestamp'].dt.month).groupby('month').count().orderBy(col('count').desc()).show(12)

In [ ]:
df_ad[['timestamp', 'action']]\
    .assign(month=lambda x: x['timestamp'].dt.month)\
    .groupby('month')\
    ['timestamp']\
    .count()\
    .nlargest(1)\
    .compute()

## 4) Nas situações onde existe um evento de view seguido de um evento de click criados pelo mesmo usuário no mesmo anúncio e campanha, quais são os 5 pares de anúncio e campanha com menores médias de tempo entre os dois eventos (2,5 pontos)

In [ ]:
df_ad.select('user_id','action',).groupBy('user_id').pivot('action').count().na.fill(0).filter(col('click')>col('view')).show()

In [ ]:
df_ad.select('user_id','action',).groupBy('user_id').pivot('action').count().na.fill(0).filter(col('click')>col('view')).count()